In [ ]:
emit = 

In [ ]:
import xarray as xr
import numpy as np
from osgeo import gdal, osr

# Path to your NetCDF file
nc_file = emit

# Output GeoTIFF filename
output_tif = 'aviris3_reflectance_gdal.tif'

# Load the dataset
ds = xr.open_dataset(nc_file)

# Extract reflectance data and transpose to (bands, rows, cols)
reflectance = ds['reflectance'].values
reflectance = np.transpose(reflectance, (2, 0, 1))  # (bands, rows, cols)

# Geotransform parameters from dataset attribute
geotransform = ds.attrs['geotransform']
origin_x = geotransform[0]
pixel_width = geotransform[1]
origin_y = geotransform[3]
pixel_height = geotransform[5]  # typically negative

# Get image dimensions
rows = ds.dims['downtrack']
cols = ds.dims['crosstrack']
bands = reflectance.shape[0]

# Create a GDAL driver for GeoTIFF
driver = gdal.GetDriverByName('GTiff')

# Create the dataset
dataset = driver.Create(output_tif, cols, rows, bands, gdal.GDT_Float32)

# Set the geotransform
dataset.SetGeoTransform([origin_x, pixel_width, 0, origin_y, 0, pixel_height])

# Set the coordinate system (assuming WGS84/latitude-longitude)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)  # or replace with your EPSG code
dataset.SetProjection(srs.ExportToWkt())

# Write each band
for i in range(bands):
    band = dataset.GetRasterBand(i + 1)
    band.WriteArray(reflectance[i])

# Close the dataset and flush to disk
dataset.FlushCache()
dataset = None

print(f"GeoTIFF created successfully: {output_tif}")
